In [1]:
import openturns as ot
import numpy as np
import otbenchmark as otb
import pandas as pd

In [2]:
p = otb.OTReliabilityAlgorithmBenchmark()
list = p.problemslist

In [3]:
k = 8

In [4]:
name = list[k].getName()
name

'RP35'

In [5]:
N = 10 ** 9
N

1000000000

In [6]:
name = list[k].getName()

resultsFORM = otb.OTReliabilityAlgorithmBenchmark.FORM(
        list[k],
        nearestPointAlgo="Cobyla",
        maximumEvaluationNumber=100,
        maximumAbsoluteError=1.0e-3,
        maximumRelativeError=1.0e-3,
        maximumResidualError=1.0e-3,
        maximumConstraintError=1.0e-3)

resultsSORM = otb.OTReliabilityAlgorithmBenchmark.SORM(
        list[k],
        nearestPointAlgo="Cobyla",
        maximumEvaluationNumber=100,
        maximumAbsoluteError=1.0e-3,
        maximumRelativeError=1.0e-3,
        maximumResidualError=1.0e-3,
        maximumConstraintError=1.0e-3)

resultsMC = otb.OTReliabilityAlgorithmBenchmark.MonteCarloSampling(list[k], maximumOuterSampling=N,
                                                                   coefficientOfVariation=0.1, blockSize=2)

resultFORMIS = otb.OTReliabilityAlgorithmBenchmark.FORMImportanceSampling(list[k],
        nearestPointAlgo="Cobyla",
        maximumEvaluationNumber=100,
        maximumAbsoluteError=1.0e-3,
        maximumRelativeError=1.0e-3,
        maximumResidualError=1.0e-3,
        maximumConstraintError=1.0e-3,
        maximumOuterSampling=N,
        coefficientOfVariation=0.1)

resultSubset = otb.OTReliabilityAlgorithmBenchmark.SubsetSampling(
        list[k], maximumOuterSampling=10000, coefficientOfVariation=0.1, blockSize=1)

result = [resultsFORM, resultsSORM, resultsMC, resultFORMIS, resultSubset]

In [7]:
a = "[%.8f,%.8f] " % ( resultsMC[4], resultsMC[5])


b = "[%.8f,%.8f] " % ( resultFORMIS[4], resultFORMIS[5])


c = "[%.8f,%.8f] " % ( resultSubset[4], resultSubset[5])


In [8]:
M = [[None]*5]*6

M[2][3] = a

M

[[None, None, None, '[0.00292795,0.00435526] ', None],
 [None, None, None, '[0.00292795,0.00435526] ', None],
 [None, None, None, '[0.00292795,0.00435526] ', None],
 [None, None, None, '[0.00292795,0.00435526] ', None],
 [None, None, None, '[0.00292795,0.00435526] ', None],
 [None, None, None, '[0.00292795,0.00435526] ', None]]

In [9]:
l = np.array([[1, 1, 1, a, 1, 1], [1, 1, 1, a, 1, 1], [1, 1, 1, a, 1, 1], [1, 1, 1, a, 1, 1],[1, 1, 1, a, 1, 1]])

In [10]:

method = ["FORM", "SORM", "CMC", "FORM-IS", "SUBSET"]
metrics = ["Probabilité exacte de "+name, "Probabilité estimée", "Erreur absolue","Intervalle de confiance",
           "nombre d'évaluations", "Log erreur relative"]

results = l



for i in range(len(method)):
    results[i][0] =  np.format_float_scientific(np.float32(list[k].getProbability()))

for i in range(len(method)):
    results[i][1] =  np.format_float_scientific(np.float32(result[i][0]))

for i in range(len(method)):
    results[i][2] =  np.format_float_scientific(np.float32(result[i][1]))

for i in range(len(method)):
    results[i][4] =  result[i][3]
    
for i in range(len(method)):
    results[i][5] =  result[i][2]

results[0][3] = "-----------------"
results[1][3] = "-----------------"
results[2][3] = a

results[3][3] = c
 
results[4][3] = b


df = pd.DataFrame(results, index = method, columns=metrics)
print("Function g : ", list[k].getEvent().getFunction())
df

Function g :  [x1,x2]->[var g1 := 2 - x2 + exp(-0.1 * x1^2) + (0.2 * x1) ^ 4;var g2 := 4.5 - x1 * x2;gsys := min(g1, g2)]


,Probabilité exacte de RP35,Probabilité estimée,Erreur absolue,Intervalle de confiance,nombre d'évaluations,Log erreur relative
FORM,3.258178e-03,1.3543355e-03,1.9038427e-03,-----------------,81,0.23334376553104713
SORM,3.258178e-03,2.1408566e-03,1.1173213e-03,-----------------,94,0.46479670739856666
CMC,3.258178e-03,3.6416072e-03,3.8342917e-04,"[0.00292795,0.00435526]",40916,0.929289662843577
FORM-IS,3.258178e-03,2.6895907e-03,5.685874e-04,"[0.00346797,0.00449803]",3388,0.7581776011706837
SUBSET,3.258178e-03,3.983e-03,7.24822e-04,"[0.00216378,0.00321540]",30000,0.652743459861216
